# PDF_doc_search:
    * This GUI can search a keyword in multiple documents. This also provide the search results based on the ranking.
    * TFIDF is used for ranking the documents.
    * PyQT5 has been used for GUI
    * PyPDF2 has been used for reading the pdf files
## Code will start with the library imports

In [1]:
import sys
from PyQt5.QtWidgets import QApplication, QHBoxLayout, QPushButton, QLineEdit, QFileDialog, QMainWindow, QGroupBox, QVBoxLayout, QFrame, QWidget, QSplitter, QLabel, QScrollArea,QFormLayout, QSlider,QHBoxLayout
from PyQt5.QtCore import Qt
import os
import pandas as pd
import PyPDF2
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from functools import reduce
from spacy.matcher import PhraseMatcher

In [2]:
class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.title = 'Keyword_finder'
        self.setWindowTitle(self.title)
        self.setGeometry(50,50,800,800)
        
        self.nlp = spacy.load('en_core_web_sm') #Smaller version of the spacy vocab has been used
        self.matcher = PhraseMatcher(self.nlp.vocab)
        self.vbox = QVBoxLayout() #Folder selection button, line editor for entering keyword and button for sentence search is laid out vertically 
        self.b1 = QPushButton('folder_name' ) # Push button for the folder selection, this will open the folder selection window
        self.b1.clicked.connect(self.file_creator)
        self.vbox.addWidget(self.b1)
        self.line = QLineEdit() # this is for entering keyword
        self.line.returnPressed.connect(self.phrase_getter) # After entering, this will check the tfidf score for 
        #all the words enter and make a sum of these so that this will get the ranking of documents based on the keyword.
        
        self.vbox.addWidget(self.line)
        self.b4 = QPushButton('sentences') # This button is used for displaying the search results.
        self.b4.clicked.connect(self.sentences)
        self.vbox.addWidget(self.b4)
        self.group = QGroupBox()
        self.group.setLayout(self.vbox)
        self.grp1 = QGroupBox()
        self.h = QHBoxLayout()
        self.s1 = QSlider(Qt.Vertical) # This slider is used for the changing the documents (rankwise stacked)
        self.s1.setMaximum(5)
        self.s1.setMinimum(0)
        self.s1.setTickInterval(1)
        self.s1.setTickPosition(0)
        self.s1.valueChanged.connect(self.sentences)
        self.h.addWidget(self.s1)
        self.s2 = QSlider(Qt.Vertical) #This slider is used for changing the sentences within a document.
        self.s2.setMaximum(10)
        self.s2.setMinimum(0)
        self.s2.setTickInterval(1)
        self.s2.setTickPosition(0)
        self.s2.valueChanged.connect(self.sentences)
        self.h.addWidget(self.s2)
        self.titl = QLabel()
        self.h.addWidget(self.titl)
        self.sent = QLabel()
        self.h.addWidget(self.sent)
        self.grp1.setLayout(self.h)
        self.splitter = QSplitter(Qt.Horizontal)
        self.frame = QFrame()
        self.splitter.addWidget(self.group)
        self.splitter.addWidget(self.grp1)
        self.hbox = QHBoxLayout()
        self.hbox.addWidget(self.splitter)
        self.setLayout(self.hbox)
        
        self.show()
        
    def file_creator(self):
        self.path = QFileDialog.getExistingDirectory()
        files = []
        self.file = []
        for i in os.listdir(self.path):
            if i.endswith('.pdf'):                            #This method is used for loading the file from local directory and
                                                              # for reading the alll documents and tokenize words as per tfidf.
                files.append(os.path.join(self.path,str(i)))
                self.file.append(str(i))
        
        
        file_data = []
        full_file_data = []
        for i in files:
            f = open(i, 'rb')
            f1 = PyPDF2.PdfFileReader(f)       
            for j in range(f1.numPages):
                page = f1.getPage(j)
                file_data.append(page.extractText())
            file_text_data = ' '.join(file_data)
            full_file_data.append(file_text_data)
            file_data = []
    
        
        self.text = pd.DataFrame()
        self.text['file'] = self.file
        self.text['content'] = full_file_data
        self.trans = TfidfVectorizer()
        self.text_transformed =self.trans.fit_transform(self.text['content'])
        self.x = self.text_transformed.toarray()
    
    def phrase_getter(self):
        self.n = self.line.text()
        self.a = self.n.split()
        self.b = [self.trans.vocabulary_[(i)] for i in self.a]
        self.p = [self.x[:,j] for j in self.b]
        self.final = (-reduce(lambda x,y: x+y , self.p)).argsort()
        self.z = []
        for i in self.final:
            doc = self.nlp(self.text['content'][i])
            self.z.append(doc)
                                                            #This method is used for ranking the documents as per the enter keywords.
                                                            #Based on the ranking, the documents are queued.
        
        phrase = self.a
        phrases = [self.nlp(text) for text in phrase]
        self.matcher.add('akash', None, *phrases)
        self.s = []
        for i in range(len(self.z)):
            matches = self.matcher(self.z[i])
            self.sentences = [self.z[i][match[1]-15: match[2]+10] for match in matches]
            self.s.append((self.file[self.final[i]],self.sentences))
            
        
        #print(self.a[0])
        #for i in self.s:
            #self.r = i
            
        return self.s 
    
    def sentences(self):
        self.r = self.phrase_getter()
        size_t = (self.s1.value())                       #Finally this is used for displaying the the searched results.
        self.titl.setText(str(self.r[size_t][0]))
        size_s = (self.s2.value())
        self.sent.setText(str(self.r[size_t][1][size_s]))

     

    

In [ ]:

app = QApplication(sys.argv)
win = Window()
sys.exit(app.exec_())